In [16]:
#    This file is part of DEAP.
#
#    DEAP is free software: you can redistribute it and/or modify
#    it under the terms of the GNU Lesser General Public License as
#    published by the Free Software Foundation, either version 3 of
#    the License, or (at your option) any later version.
#
#    DEAP is distributed in the hope that it will be useful,
#    but WITHOUT ANY WARRANTY; without even the implied warranty of
#    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
#    GNU Lesser General Public License for more details.
#
#    You should have received a copy of the GNU Lesser General Public
#    License along with DEAP. If not, see <http://www.gnu.org/licenses/>.


#    example which maximizes the sum of a list of integers
#    each of which can be 0 or 1

import random

from deap import base
from deap import creator
from deap import tools

import PyLTSpice as PyLT
from PyLTSpice import SimCommander

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

# Attribute generator
#                      define 'attr_bool' to be an attribute ('gene')
#                      which corresponds to integers sampled uniformly
#                      from the range [0,1] (i.e. 0 or 1 with equal
#                      probability)
toolbox.register("attr_resistance", random.randint, 10, 100000)

# Structure initializers
#                         define 'individual' to be an individual
#                         consisting of 100 'attr_bool' elements ('genes')
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_resistance, 2) # Voltage divider

# define the population to be a list of individuals
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# the goal ('fitness') function to be maximized
def evalVoltDivider(individual):
    if individual[0] > 0 and individual[1] > 0:
        # voltage = (10 * individual[1] / (individual[0] + individual[1]))
        LTC = SimCommander("LTFiles/ResistorDivider.asc", verbose=False)
        LTC.reset_netlist()
        LTC.set_component_value('R1', individual[0])
        LTC.set_component_value('R2', individual[1])
        LTC.add_instructions(
            "; Simulation settings",
            ".tran 0 10u 0u 10n"
        )
        LTC.run()

        LTC.wait_completion()

        LTR = PyLT.RawRead("LTFiles/ResistorDivider_1.raw", verbose=False)
        voltage = LTR.get_trace("V(vout)")[-1]
        print(f'R1: {individual[0]}, R2: {individual[1]}, Vout: {LTR.get_trace("V(vout)")[-1]}')
        return (3.85 - voltage)**2,  # 10V voltage divider target: 3.85V
    else:
        return 9999999,

#----------
# Operator registration
#----------
# register the goal / fitness function
# toolbox.register("evaluate", evalVoltDivider) # Optimizes the voltage divider problem
toolbox.register("evaluate", evalVoltDivider) # Optimizes the parallel resistance problem

# register the crossover operator
toolbox.register("mate", tools.cxTwoPoint)

# register a mutation operator with a probability to
# flip each attribute/gene of 0.05
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)

# operator for selecting individuals for breeding the next
# generation: each individual of the current generation
# is replaced by the 'fittest' (best) of three individuals
# drawn randomly from the current generation.
toolbox.register("select", tools.selTournament, tournsize=3)

C:\Users\treys\OneDrive\Documents\KU_No_Spaces\EE490\EE490_Venv\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
C:\Users\treys\OneDrive\Documents\KU_No_Spaces\EE490\EE490_Venv\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [17]:
def main():
    random.seed(64)

    # create an initial population of 300 individuals (where
    # each individual is a list of integers)
    pop = toolbox.population(n=10)

    # CXPB  is the probability with which two individuals
    #       are crossed
    #
    # MUTPB is the probability for mutating an individual
    CXPB, MUTPB = 0.5, 0.3

    print("Start of evolution")

    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    print("  Evaluated %i individuals" % len(pop))

    # Extracting all the fitnesses of
    fits = [ind.fitness.values[0] for ind in pop]

    # Variable keeping track of the number of generations
    g = 0

    # Begin the evolution
    # while max(fits) < 9.87 and g < 3000: # Voltage divider
    while g < 4:
        # A new generation
        g = g + 1
        print("-- Generation %i --" % g)

        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))

        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):

            # cross two individuals with probability CXPB
            if random.random() < CXPB:
                toolbox.mate(child1, child2)

                # fitness values of the children
                # must be recalculated later
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:

            # mutate an individual with probability MUTPB
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        print("  Evaluated %i individuals" % len(invalid_ind))

        # The population is entirely replaced by the offspring
        pop[:] = offspring

        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in pop]

        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5

        print("  Min %s" % min(fits))
        print("  Max %s" % max(fits))
        print("  Avg %s" % mean)
        print("  Std %s" % std)

    print("-- End of (successful) evolution --")

    best_ind = tools.selBest(pop, 1)[0]
    print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))

In [18]:
if __name__ == "__main__":
    main()

Start of evolution
Using LTspice installed in : 'C:\Program Files\LTC\LTspiceXVII\XVIIx64.exe' 
R1: 62421, R2: 16374, Vout: 2.0780506134033203
Using LTspice installed in : 'C:\Program Files\LTC\LTspiceXVII\XVIIx64.exe' 
R1: 82614, R2: 80355, Vout: 4.930692195892334
Using LTspice installed in : 'C:\Program Files\LTC\LTspiceXVII\XVIIx64.exe' 
R1: 52957, R2: 70285, Vout: 5.703007221221924
Using LTspice installed in : 'C:\Program Files\LTC\LTspiceXVII\XVIIx64.exe' 
R1: 90038, R2: 2218, Vout: 0.2404179722070694
Using LTspice installed in : 'C:\Program Files\LTC\LTspiceXVII\XVIIx64.exe' 
R1: 26684, R2: 35222, Vout: 5.68959379196167
Using LTspice installed in : 'C:\Program Files\LTC\LTspiceXVII\XVIIx64.exe' 
R1: 92299, R2: 82919, Vout: 4.732333660125732
Using LTspice installed in : 'C:\Program Files\LTC\LTspiceXVII\XVIIx64.exe' 
R1: 26403, R2: 91584, Vout: 7.762211322784424
Using LTspice installed in : 'C:\Program Files\LTC\LTspiceXVII\XVIIx64.exe' 
R1: 85595, R2: 19334, Vout: 1.8425792455673